<img src="images/google-breakout.jpg" class="img-responsive img-rounded" style="width:100%">
Part 2 에서는 DeepMind 팀에서 내놓은 Playing Atari with Deep Reinforcement Learning 논문을 해부할 것입니다.

In [14]:
import pylab
import numpy as np

### 1. Introduction

이미지 또는 음성등에서 바로 Agent를 학습시키는 것은 RL (Reinforcement Learning)에서 오래된 챌린지중의 하나입니다.<br>
이전의 RL방식들은 손으로 집접 만든 features들이나 policy등을 통해서 성공할수 있었지만 이 경우 특정 문제를 해결하는데만 최적화가 되어 있어 <br>
같은 방식으로 다른 문제들을 해결하기에는 어려운 점들이 많습니다.

최근 Deep Learning의 발전들은 raw sensory data (이미지등)에서 high-level features들을 뽑아내는게 가능하게 만들었고,<br>
이는 Convolutional networks, Multiplayer Perceptrons, restricted Boltzmann machines 그리고 
Recurrent Neural networks와 같은  컴퓨터 비전[11, 22, 16] 그리고 음성인식 [6, 7]에서의 비약적인 발전으로 이어졌습니다. 

하지만 Reinforcement Learning 은 deep learning의 관점에서 볼때 여러 챌린지들을 갖고 있습니다.<br>
첫번째로 성공적인 deep learning applications들은 수작업한 엄청나게 많은 데이터를 통해서 학습됩니다. 
하지만 RL 알고리즘은 그와는 반대로 scalar reward signal을 통해서 배워야만 하며, 
이 reward는 매우 적게 분포하고 있으며 (frequently sparse), delayed 된 경우가 많습니다.  
실질적으로 delay는 actions과 resulting rewards 사이에 수천 timesteps이 존재할정도로 거대합니다. 
이는 기존의 input과 targets이 direct로 연견될것과 비교해볼수 있습니다.<br>

다른 이슈는 기존의 대부분의 deep learning이 모든 samples들이 independent 하다고 여깁니다.<br> 
하지만 Deep Reinforcement Learning 에서는 매우 연관성이 높은 (correrated) states의 sequences를 만나게 될 일이 많습니다.

### Background


environment $ \epsilon $에 해당하는 Atari emulator안에서 Agent는 일련의 actions, observations, 그리고 rewards등을 받습니다.<br>
각각의 time-step마다 Agent는 게임안에서 허용된 $ A = \{1, ..., K\} $ actions들로 부터 하나의 $ a_{t} $ (action)을 취하게 됩니다.<br>
Action은 environment (Atari Emulator)안으로 들어가게 되고, Game state, score가 변하게 됩니다. 
Emulator안의 내부 state자체를 Agent가 얻는것이 아니라 (예를 들어서 공 object의 위치나, paddle object의 위치 등등) 이미지 자체를 $ x_t \in \Bbb{R}^d $
(x 값이라는 것은.. input data로 사용된다는 뜻이고 x_t 라는건 어느 시점 (time)의 input image data를 말함) 

이미지 한장을 보고서 Agent가 어떤 상황인지 알아내는 것은 불가능합니다.<br>
따라서 일련의 actions 그리고 화면이미지들 (observations) $ s_t= x_1,a_1,x_2,...,a_{t-1},x_t, $ 을 통해서 학습을 합니다.<br>
모든 게임은 한정된 time-steps를 지나고 끝나게 됩니다. 따라서 MDPs (Markov Decision Process)를 사용해서 문제를 해결할 수 있습니다.


| Name | Math Symbol | Description |
|:-----|:------------|:------------|
| Environment | $$ \varepsilon $$ | Atari Emulator 를 뜻하며 Agent는 environment로 부터 actions, observations, rewards등을 주거니 받거니함<br>일반적으로  stochastic. |
| Action      | $$ a_t $$ | 특정 시점의 action을 말하며, 게임안에서 허용된 $$ A = \{1, ..., K\} $$ 중에 하나를 사용함 |
| Image (screen shot) | $$ x_t \in \Bbb{R}^d $$ | 현재시점의 화면을 나타내는 이미지 |
| Reward | $$ r_t $$ | Reward는 이전 **전체** actions 그리고 observations과 연관이 있음. <br>즉 하나의 action에 대한 feedback을 받으려면 일반적으로 수천번의 time-steps이 지나간 이후 받을수 있음  |   




**Agent의 목표**는 미래에 받게될 rewards의 양을 최대한 늘리는 방향으로 actions을 선택하는 것입니다.<br>
미래에 받게될 보상은 time-step 마다 a factor of $ \gamma $ 만큼 줄어든다고 가정을 합니다. (미래는 불확실하기 때문에) <br>
특정시점 $ t $ 에 받게될 future discounted return은 다음과 같이 정의 합니다.

$$ R_t = \sum^{T}_{t^{\prime} = t} \gamma^{t^{\prime} - t} r $$

| Name | Math Symbol | Description |
|:-----|:------------|:------------|
| 종료 시점 | $ T $ | 게임이 종료되는 시점 |
| 현재 시점 | $ t $ | 계산을 하려는 시점 (현재 시점) |


In [27]:
# 예를 들어서 도박으로 매달 100만원 정도씩 벌 수 있는 친구가 있습니다. 
# discounted future return으로 6개월 동안 도박을 했을때 벌어들일수 있는 전체 수익은 얼마일까요?
# 이때 이 친구의 실력이 꽤나 뛰어나서 계속 100만원을 벌수 있는 확률을 90%정도로 보겠습니다. 

def discounted_future_return(gamma, reward, T):
    net_reward = 0
    for t in xrange(T):
        net_reward += gamma**t * reward
        print '%d = %.1f^%d * %d' % (net_reward, gamma, t, reward)
    return net_reward
        
discounted_future_return(gamma=0.9, reward=100, T=6)

100 = 0.9^0 * 100
190 = 0.9^1 * 100
271 = 0.9^2 * 100
343 = 0.9^3 * 100
409 = 0.9^4 * 100
468 = 0.9^5 * 100


468.55899999999997

### References 

* [Playing Atari with Deep Reinforcement Learning](https://www.cs.toronto.edu/~vmnih/docs/dqn.pdf)